In [23]:
from biocrnpyler import *

from biocrnpyler.mechanism import SimpleTranscription, Translation_MM, SimpleTranslation
from biocrnpyler.global_mechanism import Dilution
from biocrnpyler.component import Protein
from biocrnpyler import Mechanism
from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork

import numpy as np

import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
bokeh.io.output_notebook()

Loading BokehJS ...

Hierarchy: Mixture $\to$ Component $\to$ Mechanism $\to$ Species and Reactions $\to$ Compiled CRN

Remember overarching goal: Want extended lifetime of protein production (TX/TL) in synthetic cells. We will do this by looking into mechanisms for ATP regeneration.

**For ATP regeneration machinery without Tx/Tl**
- Mixture
    - Cell Extract which includes glucose, atp, any necessary enzymes
- Components (?)
    - enzyme complexes, enzymes, all species
    - wouldn't include enzyme complexes?
        - use ComplexSpecies constructor to generated complexes between species
- Mechanisms
    - want glucose $\to$ f16p $\to$ isobutanol
    - and atp $\to$ adp+pi - degradation
    - modeled as mass action for now
    - incorporate different enzyme models here
    - transcription/translation (?)
- Parameters
    - kf = 10
    - kr = 0.1
    - delta = 10
- Species
    - atp, glucose, adp, f16p, isobutanol, pi, and intermediate complexes

Remember the simplified model

**Combined**
$$\text{2 ATP + Glucose} \leftrightarrow^{E1} \text{2 ADP + F16P}$$
**Non-ATP Generating**
$$\text{2 ADP + F16P} \leftrightarrow^{E2} \text{2 ATP + Isobutanol}$$
**ATP Generating**
$$\text{3 ADP + Pi + F16P} \leftrightarrow^{E3} \text{3 ATP + Isobutanol}$$
**Spontaneous Hydrolysis**
$$\text{ATP} \rightarrow \text{ADP + Pi}$$




## Incorporating Complexes

In [28]:
# Define the species in the CRN
ATP = Species(name = 'ATP', material_type = 'metabolite')
ADP = Species(name = "ADP", material_type = 'metabolite')
Pi = Species(name = "Pi", material_type = 'metabolite')

Glucose = Species(name = 'Glucose', material_type = 'sugar')
F16P = Species(name = 'F16P', material_type = 'sugar')
Isobutanol = Species(name = 'Isobutanol', material_type = 'sugar_product')

E1 = Species(name = 'E1', material_type = 'enzyme')
E2 = Species(name = 'E2', material_type = 'enzyme')
E3 = Species(name = 'E3', material_type = 'enzyme')

Glucose_2ATP_E1 = ComplexSpecies([Glucose, ATP, ATP, E1])
F16P_2ADP_E1 = ComplexSpecies([F16P, ADP, ADP, E1])

F16P_2ADP_E2 = ComplexSpecies([F16P, ADP, ADP, E2])
Isobutanol_2ATP_E2 = ComplexSpecies([Isobutanol, ATP, ATP, E2])

F16P_3ADP_Pi_E3 = ComplexSpecies([F16P, ADP, ADP, ADP, Pi, E3])
Isobutanol_3ATP_E3 = ComplexSpecies([Isobutanol, ATP, ATP, ATP, E3])

species = [ATP, ADP, Pi, Glucose, F16P, Isobutanol, E1, E2, E3, Glucose_2ATP_E1, F16P_2ADP_E1,
          F16P_2ADP_E2, Isobutanol_2ATP_E2, F16P_3ADP_Pi_E3, Isobutanol_3ATP_E3]

# Reaction rates
kf = 10
kr = 0.1

# Combined rxn 1 (reversible)
inputs1 = [ATP, ATP, Glucose, E1]
outputs1 = [Glucose_2ATP_E1]
rxn1 = Reaction(inputs1, outputs1, k = kf, k_rev = kr)

# Combined rxn 2 (irreversible)
inputs2 = [Glucose_2ATP_E1]
outputs2 = [F16P_2ADP_E1]
kexpress = 10.
rxn2 = Reaction(inputs2, outputs2, k = kexpress)

# Combined rxn 3
inputs3 = [F16P_2ADP_E1]
outputs3 = [ADP, ADP, F16P]
rxn3 = Reaction(inputs3, outputs3, k = kf, k_rev = kr)

# NON rxn 4
inputs4 = [ADP, ADP, F16P, E2]
outputs4 = [F16P_2ADP_E2]
rxn4 = Reaction(inputs4, outputs4, k = kf, k_rev = kr)

# NON rxn 5
inputs5 = [F16P_2ADP_E2]
outputs5 = [Isobutanol_2ATP_E2]
rxn5 = Reaction(inputs5, outputs5, k = kexpress)

# NON rxn 6
inputs6 = [Isobutanol_2ATP_E2]
outputs6 = [Isobutanol, ATP, ATP, E2]
rxn6 = Reaction(inputs6, outputs6, k = kf, k_rev = kr)

# ATP rxn 7
inputs7 = [ADP, ADP, Pi, F16P, E3]
outputs7 = [F16P_3ADP_Pi_E3]
rxn7 = Reaction(inputs7, outputs7, k = kf, k_rev = kr)

# ATP rxn 8
inputs8 = [F16P_3ADP_Pi_E3]
outputs8 = [Isobutanol_3ATP_E3]
rxn8 = Reaction(inputs8, outputs8, k = kexpress)

# ATP rxn 9
inputs9 = [Isobutanol_3ATP_E3]
outputs9 = [Isobutanol, ATP, ATP, ATP, E3]
rxn9 = Reaction(inputs9, outputs9, k = kf, k_rev = kr)

rxns = [rxn1, rxn2, rxn3, rxn4, rxn5, rxn6, rxn7, rxn8, rxn9]

CRN = ChemicalReactionNetwork(species, rxns)

In [29]:
#Expression Extract is a Mixture with very simple internal models
Model_expression = ExpressionExtract(name = "extract", species = [Glucose, ATP, E1, E2, E3, ADP, Pi, F16P, Isobutanol,
                                                                 Glucose_2ATP_E1, F16P_2ADP_E1, F16P_2ADP_E2,
                                                                 Isobutanol_2ATP_E2, F16P_3ADP_Pi_E3,Isobutanol_3ATP_E3 ],
                                     reactions = rxns,
                                     parameter_file = "parameters.txt")

#print("BioCRNpyler Representation:\n", repr(Model_expression))

#BioCRNpyler Models are Compiled into a CRN
CRN_expression = Model_expression.compile_crn()
print("\nCRN Representation:\n", repr(CRN_expression))

#Save as SBML
CRN_expression.write_sbml_file("CRN_expression.sbml")

#Simulate with Bioscrape
timepoints = np.linspace(0, 200, 1000)
x0 = {"dna_GFP":1.0}
Rexpress, mexpress = CRN_expression.simulate_with_bioscrape_via_sbml(timepoints, initial_condition_dict = x0, file = "CRN_expression.sbml")


CRN Representation:
 Species = sugar_Glucose, metabolite_ATP, enzyme_E1, enzyme_E2, enzyme_E3, metabolite_ADP, metabolite_Pi, sugar_F16P, sugar_product_Isobutanol, complex_metabolite_ATP_metabolite_ATP_enzyme_E1_sugar_Glucose, complex_metabolite_ADP_metabolite_ADP_enzyme_E1_sugar_F16P, complex_metabolite_ADP_metabolite_ADP_enzyme_E2_sugar_F16P, complex_metabolite_ATP_metabolite_ATP_enzyme_E2_sugar_product_Isobutanol, complex_metabolite_ADP_metabolite_ADP_metabolite_ADP_enzyme_E3_sugar_F16P_metabolite_Pi, complex_metabolite_ATP_metabolite_ATP_metabolite_ATP_enzyme_E3_sugar_product_Isobutanol
Reactions = [
]


/anaconda3/lib/python3.7/site-packages/bioscrape/sbmlutil.py:209: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +
/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/chemical_reaction_network.py:664: UserWarning: Trying to set species that is not in model: dna_GFP
  m.set_species(initial_condition_dict)


In [27]:
Rexpress

,sugar_Glucose,metabolite_ATP,enzyme_E1,enzyme_E2,enzyme_E3,metabolite_ADP,metabolite_Pi,sugar_F16P,sugar_product_Isobutanol,complex_metabolite_ATP_metabolite_ATP_enzyme_E1_sugar_Glucose,complex_metabolite_ADP_metabolite_ADP_enzyme_E1_sugar_F16P,complex_metabolite_ADP_metabolite_ADP_enzyme_E2_sugar_F16P,complex_metabolite_ATP_metabolite_ATP_enzyme_E2_sugar_product_Isobutanol,complex_metabolite_ADP_metabolite_ADP_metabolite_ADP_enzyme_E3_sugar_F16P_metabolite_Pi,complex_metabolite_ATP_metabolite_ATP_metabolite_ATP_enzyme_E3_sugar_product_Isobutanol,time
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200200
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400400
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.600601
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,199.199199
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,199.399399
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,199.599600
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,199.799800


In [25]:
# First plot glucose and isobutanol
colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f']
p1 = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p1.line(timepoints, Rexpress['sugar_Glucose'], color = colors[0], legend_label = 'glucose')
p1.line(timepoints, Rexpress['sugar_F16P'], color = colors[1], legend_label = 'f16p')
p1.line(timepoints, Rexpress['sugar_product_Isobutanol'], color = colors[2], legend_label = 'isobutanol')
p1.legend.location = 'center_right'

p2 = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p2.line(timepoints, Rexpress['metabolite_ATP'], color = colors[3], legend_label = 'atp')
p2.line(timepoints, Rexpress['metabolite_ADP'], color = colors[4], legend_label = 'adp')
p2.line(timepoints, Rexpress['metabolite_Pi'], color = colors[5], legend_label = 'pi')
p2.legend.location = 'center_right'


# Show plots
bokeh.io.show(row(p1, p2))

## Define custom mixture
Just has ATP, Glucose, and Enzymes.


In [14]:
class ExpressionMixture(Mixture):
    def __init__(self, name = '', ATP = 'ATP', Glucose = 'Glucose', **keywords):
        if isinstance(ATP, Species):
            self.atp = atp
        elif isinstance(ATP, str):
            self.atp = Species(name = ATP, material_type = 'metabolite')
        elif isinstance(ATP, Component) and atp.get_species() != None:
            self.atp = ATP.get_species()
        else:
            raise ValueError('atp error')
        
            
        if isinstance(Glucose, Species):
            self.glucose = Glucose
        elif isinstance(Glucose, str):
            self.glucose = Species(name = Glucose, material_type = 'metabolite')
        elif isinstance(Glucose, Component) and Glucose.get_species() != None:
            self.glucose = Glucose.get_species()
        else:
            raise ValueError('glucose error')
        
        

In [ ]:
class DilutionMixture(Mixture):
    def __init__(self, name = '', **keywords):
        

## Define custom mechanisms
3 enzymatic models

In [32]:
class EnzymaticModel1(Mechanism):
    # Set the name and mechanism_type
    def __init__(self, name='enzymatic_model_1', mechanism_type = 'enzyme'):
        Mechanism.__init__(self, name = name, mechanism_type = mechanism_type)
        
    # Overwrite update species
    def update_species(self, fuel, substrate, enzyme, waste, product, **keywords):
        # Define all complexes
        # as 1 step MM
        return [fuel, substrate, enzyme, waste, product]
    
    def update_reactions(self, fuel, substrate, enzyme, waste, product, component, part_id, **keywords):
        
        # model all enzymatic reactions here
        
        kf = component.get_parameters("kf", part_id = part_id, mechanism = self)

        params = {'kf': kf}
        
        reaction = Reaction(inputs = [fuel, substrate, enzyme], output = [waste, product, enzyme],
                           propensity_type = 'massaction', propensity_params = params)
        
        return [reaction]